<a href="https://colab.research.google.com/github/ag4267research1/Solving-Poisson-s-Equation-with-GNO/blob/main/GNOPDE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install neuraloperator --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.6/248.6 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 112.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.3/59.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.2/788.2 kB 72.7 MB/s eta 0:00:00


In [3]:
import os, pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split

# Correct imports:
from neuralop.layers.gno_block import GNOBlock
from neuralop.data.transforms.normalizers import UnitGaussianNormalizer
from neuralop.training import Trainer



In [5]:
# Folder in the Colab VM (not Drive)
!mkdir -p /content/nonlinear_poisson
!cd /content/nonlinear_poisson

# Download with progress bar (percentage + speed)
!wget --show-progress \
  "https://zenodo.org/records/15001788/files/nonlinear_poisson.obj?download=1" \
  -O nonlinear_poisson.obj




--2025-12-07 18:59:03--  https://zenodo.org/records/15001788/files/nonlinear_poisson.obj?download=1
Resolving zenodo.org (zenodo.org)... 188.185.48.75, 137.138.52.235, 188.185.43.153, ...
Connecting to zenodo.org (zenodo.org)|188.185.48.75|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9608410281 (8.9G) [application/octet-stream]
Saving to: ‘nonlinear_poisson.obj’

nonlinear_poisson.o 100%[===================>]   8.95G  19.1MB/s    in 8m 2s   

2025-12-07 19:07:05 (19.0 MB/s) - ‘nonlinear_poisson.obj’ saved [9608410281/9608410281]



In [12]:
# Show file size
!ls -lh nonlinear_poisson.obj

-rw-r--r-- 1 root root 9.0G Dec  7 19:07 nonlinear_poisson.obj


In [15]:
DATA_PATH = "/content/nonlinear_poisson.obj"
print("Exists?", os.path.exists(DATA_PATH))

Exists? True


In [16]:
with open(DATA_PATH, "rb") as f:
    data = pickle.load(f)

print(type(data))
print(len(data))

<class 'list'>
10000


In [17]:
sample = data[0]
print(type(sample))
print(sample.keys())

<class 'dict'>
dict_keys(['train_points_boundary', 'train_values_boundary', 'train_source_terms_boundary', 'train_bc_boundary', 'train_points_domain', 'train_values_domain', 'train_distances_domain', 'train_source_terms_domain', 'train_bc_domain', 'val_points_boundary', 'val_values_boundary', 'val_source_terms_boundary', 'val_points_domain', 'val_values_domain', 'val_source_terms_domain', 'coefs'])


In [18]:
class NonlinearPoissonDataset(Dataset):
    def __init__(self, data_list):
        self.data = data_list

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]

        # DOMAIN points (P, 2)
        coords = torch.tensor(sample["train_points_domain"], dtype=torch.float32)

        # FORCING (P, 1)
        forcing = torch.tensor(sample["train_source_terms_domain"], dtype=torch.float32)

        # SOLUTION u(x) (P, 1)
        solution = torch.tensor(sample["train_values_domain"], dtype=torch.float32)

        # OPTIONAL: include distance-to-boundary
        # dist = torch.tensor(sample["train_distances_domain"], dtype=torch.float32)
        # forcing = torch.cat([forcing, dist], dim=-1)

        return coords, forcing, solution

In [19]:
dataset = NonlinearPoissonDataset(data)
coords, f, u = dataset[0]

print(coords.shape)
print(f.shape)
print(u.shape)

torch.Size([10000, 2])
torch.Size([10000])
torch.Size([10000])


# MODEL

In [4]:
class GNOPoissonModel(nn.Module):
    def __init__(self, in_channels=1, out_channels=1, coord_dim=2,
                 hidden_channels=64, radius=0.2, n_layers=3):
        super().__init__()

        layers = []
        c_in = in_channels
        for _ in range(n_layers):
            layers.append(
                GNOBlock(
                    in_channels=c_in,
                    out_channels=hidden_channels,
                    coord_dim=coord_dim,
                    radius=radius,
                    # keep defaults: linear transform_type, sum reduction, transformer PE, etc.
                )
            )
            c_in = hidden_channels

        self.gno_layers = nn.ModuleList(layers)
        self.out_layer = nn.Linear(hidden_channels, out_channels)

    def forward(self, x, coords):
        """
        x:      (B, P, C_in)
        coords: (B, P, coord_dim)
        """
        h = x
        for gno in self.gno_layers:
            h = gno(h, coords)
            h = F.gelu(h)

        # pointwise readout at each node
        out = self.out_layer(h)
        return out